In [ ]:
import json
from pathlib import Path
import pandas as pd
from rapidfuzz import process, fuzz
from tqdm import tqdm

# ---------------- CONFIG ----------------
GENE_LIST_PATH = "**/Data_processing/Data_processing_output/all_genes480_name.txt"

PANGLOA_FILE = "**/Data/PanglaoDB_markers_27_Mar_2020.tsv.gz"
CELLMARKER_FILE = "**/Data/Cell_marker_Human.xlsx"  # 

OUT_DIR = Path("Data_processing_output/RAG_output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_JSON = OUT_DIR / "gene_knowledge.json"
OUT_REVIEW = OUT_DIR / "gene_knowledge.csv"

# matching parameters
FUZZY_THRESHOLD = 85   # adjust (80-90) if needed
PREFERENCE_ORDER = ["PanglaoDB", "CellMarker"]  # which DB to prefer when both provide info
# ----------------------------------------

print("Config OK.")
print("Gene list:", GENE_LIST_PATH)
print("Panglao file:", PANGLOA_FILE)
print("CellMarker file:", CELLMARKER_FILE)
print("Outputs to:", OUT_DIR.resolve())


In [3]:
def load_gene_list(path):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Gene list not found: {p}")
    if p.suffix.lower() == ".txt":
        return [l.strip() for l in p.read_text(encoding='utf-8').splitlines() if l.strip()]
    if p.suffix.lower() == ".json":
        return json.loads(p.read_text(encoding='utf-8'))
    # csv fallback
    df = pd.read_csv(p, dtype=str)
    if "gene" in [c.lower() for c in df.columns]:
        col = [c for c in df.columns if c.lower()=="gene"][0]
        return df[col].astype(str).tolist()
    return df.iloc[:,0].astype(str).tolist()

genes = load_gene_list(GENE_LIST_PATH)
print("Loaded genes:", len(genes))
genes[:10]


Loaded genes: 480


['ABCC4',
 'ACE2',
 'ACSL5',
 'ADAM12',
 'ADH1A',
 'ADH4',
 'ADH6',
 'AFF3',
 'AFM',
 'AGR2']

In [4]:
# === Load PanglaoDB (tsv.gz) ============================
panglao = pd.read_csv(PANGLOA_FILE, sep="\t", compression="gzip", dtype=str).fillna("")
print("Panglao shape:", panglao.shape)
print("Panglao columns:", panglao.columns.tolist())
display(panglao.head())

# === Load CellMarker (xlsx only) ========================
cm_path = Path(CELLMARKER_FILE)

if not cm_path.exists():
    raise FileNotFoundError(f"CellMarker file not found: {cm_path}")

# Since you have .xlsx file → directly read_excel()
cellmarker = pd.read_excel(cm_path, dtype=str).fillna("")

print("\nCellMarker shape:", cellmarker.shape)
print("CellMarker columns:", cellmarker.columns.tolist())
display(cellmarker.head())




Panglao shape: (8286, 14)
Panglao columns: ['species', 'official gene symbol', 'cell type', 'nicknames', 'ubiquitousness index', 'product description', 'gene type', 'canonical marker', 'germ layer', 'organ', 'sensitivity_human', 'sensitivity_mouse', 'specificity_human', 'specificity_mouse']


,species,official gene symbol,cell type,nicknames,ubiquitousness index,product description,gene type,canonical marker,germ layer,organ,sensitivity_human,sensitivity_mouse,specificity_human,specificity_mouse
0,Mm Hs,CTRB1,Acinar cells,CTRB,0.017,chymotrypsinogen B1,protein-coding gene,1,Endoderm,Pancreas,1.0,0.957143,0.000628931,0.0159201
1,Mm Hs,KLK1,Acinar cells,Klk6,0.013,kallikrein 1,protein-coding gene,1,Endoderm,Pancreas,0.833333,0.314286,0.00503145,0.0128263
2,Mm Hs,RBPJL,Acinar cells,RBP-L|SUHL|RBPSUHL,0.001,recombination signal binding protein for immun...,protein-coding gene,1,Endoderm,Pancreas,0.0,0.0,0.0,0.0
3,Mm Hs,PTF1A,Acinar cells,PTF1-p48|bHLHa29,0.001,pancreas associated transcription factor 1a,protein-coding gene,1,Endoderm,Pancreas,0.0,0.157143,0.000628931,0.000773445
4,Mm,TRY4,Acinar cells,,0.007,trypsin 4,protein coding gene,1,Endoderm,Pancreas,,,,



CellMarker shape: (60877, 20)
CellMarker columns: ['species', 'tissue_class', 'tissue_type', 'uberonongology_id', 'cancer_type', 'cell_type', 'cell_name', 'cellontology_id', 'marker', 'Symbol', 'GeneID', 'Genetype', 'Genename', 'UNIPROTID', 'technology_seq', 'marker_source', 'PMID', 'Title', 'journal', 'year']


,species,tissue_class,tissue_type,uberonongology_id,cancer_type,cell_type,cell_name,cellontology_id,marker,Symbol,GeneID,Genetype,Genename,UNIPROTID,technology_seq,marker_source,PMID,Title,journal,year
0,Human,Abdomen,Abdomen,UBERON_0000916,Normal,Normal cell,Macrophage,CL_0000235,MERTK,MERTK,10461,protein_coding,"MER proto-oncogene, tyrosine kinase",Q12866,,Experiment,31982413,Peritoneal Level of CD206 Associates With Mort...,Gastroenterology,2020
1,Human,Abdomen,Abdomen,UBERON_0000916,Normal,Normal cell,Macrophage,CL_0000235,CD16,FCGR3A,2215,protein_coding,Fc fragment of IgG receptor IIIb,O75015,,Experiment,31982413,Peritoneal Level of CD206 Associates With Mort...,Gastroenterology,2020
2,Human,Abdomen,Abdomen,UBERON_0000916,Normal,Normal cell,Macrophage,CL_0000235,CD206,MRC1,4360,protein_coding,mannose receptor C-type 1,P22897,,Experiment,31982413,Peritoneal Level of CD206 Associates With Mort...,Gastroenterology,2020
3,Human,Abdomen,Abdomen,UBERON_0000916,Normal,Normal cell,Macrophage,CL_0000235,CRIg,VSIG4,11326,protein_coding,V-set and immunoglobulin domain containing 4,Q9Y279,,Experiment,31982413,Peritoneal Level of CD206 Associates With Mort...,Gastroenterology,2020
4,Human,Abdomen,Abdomen,UBERON_0000916,Normal,Normal cell,Macrophage,CL_0000235,CD163,CD163,9332,protein_coding,CD163 molecule,Q86VB7,,Experiment,31982413,Peritoneal Level of CD206 Associates With Mort...,Gastroenterology,2020


In [5]:
# For your Panglao file (from earlier inspection we expect these)
P_COL_SPECIES = "species"
P_COL_SYMBOL  = "official gene symbol"
P_COL_CELL    = "cell type"
P_COL_ALIAS   = "nicknames"
P_COL_DESC    = "product description"

# print("Panglao columns used:", P_COL_SPECIES, P_COL_SYMBOL, P_COL_CELL, P_COL_ALIAS, P_COL_DESC)
panglao[[P_COL_SPECIES, P_COL_SYMBOL, P_COL_CELL, P_COL_ALIAS, P_COL_DESC]].head()



,species,official gene symbol,cell type,nicknames,product description
0,Mm Hs,CTRB1,Acinar cells,CTRB,chymotrypsinogen B1
1,Mm Hs,KLK1,Acinar cells,Klk6,kallikrein 1
2,Mm Hs,RBPJL,Acinar cells,RBP-L|SUHL|RBPSUHL,recombination signal binding protein for immun...
3,Mm Hs,PTF1A,Acinar cells,PTF1-p48|bHLHa29,pancreas associated transcription factor 1a
4,Mm,TRY4,Acinar cells,,trypsin 4


In [6]:
# For CellMarker, inspect and set appropriate columns.
# Common column names vary — set these to match your CellMarker CSV.
# Example guesses (update these if your CellMarker columns differ):
C_COL_SPECIES = "species"        # to filter Hs rows
C_COL_SYMBOL  = "Symbol"         # official gene symbol
C_COL_CELL    = "cell_type"      # the actual cell type
C_COL_DESC    = "Genename"       # clean gene name (best available description)
C_COL_ALIAS   = None             # no alias column in your file


cellmarker[[C_COL_SPECIES, C_COL_SYMBOL, C_COL_CELL, C_COL_DESC]].head()

,species,Symbol,cell_type,Genename
0,Human,MERTK,Normal cell,"MER proto-oncogene, tyrosine kinase"
1,Human,FCGR3A,Normal cell,Fc fragment of IgG receptor IIIb
2,Human,MRC1,Normal cell,mannose receptor C-type 1
3,Human,VSIG4,Normal cell,V-set and immunoglobulin domain containing 4
4,Human,CD163,Normal cell,CD163 molecule


In [7]:
# ==================================================
# Build lookups for PanglaoDB (human rows) and CellMarker (human)
# ==================================================

# --- REQUIRED: ensure these variables already set earlier in your notebook ---
# P_COL_SPECIES, P_COL_SYMBOL, P_COL_ALIAS
# C_COL_SPECIES, C_COL_SYMBOL
# panglao (DataFrame), cellmarker (DataFrame) already loaded

# If C_COL_ALIAS not present in CellMarker, keep it None
C_COL_ALIAS = None

# 1) Filter Panglao to include human (Hs) or Mm Hs
if P_COL_SPECIES in panglao.columns:
    panglao_h = panglao[panglao[P_COL_SPECIES].astype(str).str.contains("Hs", case=False, na=False)].copy()
else:
    panglao_h = panglao.copy()
print("Panglao human rows:", panglao_h.shape)

# 2) Generic lookup builder (handles symbol and alias columns)
def build_lookup_from_df(df, symbol_col, alias_col=None):
    lookup = {}
    if symbol_col not in df.columns:
        raise KeyError(f"Symbol column '{symbol_col}' not found in DataFrame columns: {list(df.columns)[:10]}")
    for _, row in df.iterrows():
        rd = row.to_dict()
        # official symbol
        s = str(rd.get(symbol_col,"")).strip()
        if s:
            lookup.setdefault(s.lower(), []).append(("symbol", rd))
        # aliases (optional)
        if alias_col and alias_col in rd and rd.get(alias_col):
            alias_field = str(rd.get(alias_col,""))
            tokens = [t.strip() for t in alias_field.replace("|",",").replace(";",",").split(",")]
            for t in tokens:
                if t:
                    lookup.setdefault(t.lower(), []).append(("alias", rd))
    return lookup

# 3) Build Panglao lookup
panglao_lookup = build_lookup_from_df(panglao_h, P_COL_SYMBOL, P_COL_ALIAS)
print("Panglao lookup tokens:", len(panglao_lookup))

# 4) Build CellMarker lookup (filter to Human rows if species col exists)
cellmarker_lookup = {}
if cellmarker is None or cellmarker.empty:
    print("CellMarker dataframe is empty or not loaded.")
else:
    if C_COL_SPECIES in cellmarker.columns:
        cm_h = cellmarker[cellmarker[C_COL_SPECIES].astype(str).str.contains("Hs|Human", case=False, na=False)].copy()
        print("CellMarker human-like rows:", cm_h.shape)
    else:
        cm_h = cellmarker.copy()
        print("CellMarker rows (no species filter):", cm_h.shape)
    # ensure symbol column exists
    if C_COL_SYMBOL not in cm_h.columns:
        raise KeyError(f"CellMarker symbol column '{C_COL_SYMBOL}' not found. Available columns: {list(cm_h.columns)[:20]}")
    # build lookup (CellMarker in your file has no alias column)
    cellmarker_lookup = build_lookup_from_df(cm_h, C_COL_SYMBOL, alias_col=None)
    print("CellMarker lookup tokens:", len(cellmarker_lookup))

# 5) Quick sanity samples (show some lookup keys)
print("\nSample Panglao tokens (first 20):", list(panglao_lookup.keys())[:20])
print("Sample CellMarker tokens (first 20):", list(cellmarker_lookup.keys())[:20])


Panglao human rows: (7851, 14)
Panglao lookup tokens: 13717
CellMarker human-like rows: (60877, 20)
CellMarker lookup tokens: 12936

Sample Panglao tokens (first 20): ['ctrb1', 'ctrb', 'klk1', 'klk6', 'rbpjl', 'rbp-l', 'suhl', 'rbpsuhl', 'ptf1a', 'ptf1-p48', 'bhlha29', 'cela3a', 'ela3', 'ela3a', 'prss1', 'try1', 'spink1', 'spink3', 'pctt', 'psti']
Sample CellMarker tokens (first 20): ['mertk', 'fcgr3a', 'mrc1', 'vsig4', 'cd163', 'fabp4', 'ucp1', 'cd19', 'cd34', 'cd44', 'cidea', 'zic1', 'dlk1', 'bscl2', 'adipor1', 'dlk2', 'adipor2', 'vstm2a', 'vstm2b', 'alcam']


In [9]:
from tqdm import tqdm
import json
import pandas as pd
from pathlib import Path

OUT_DIR = Path("Data_processing_output/RAG_output")
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_JSON = OUT_DIR / "gene_knowledge.json"
OUT_REVIEW = OUT_DIR / "gene_knowledge.csv"

def extract_short_desc_from_row(rowdict, desc_col, cell_col):
    if desc_col and rowdict.get(desc_col):
        return str(rowdict[desc_col]).strip()
    if cell_col and rowdict.get(cell_col):
        return str(rowdict[cell_col]).strip()
    return ""

def match_in_lookup(gene, lookup):
    key = gene.lower().strip()
    if key in lookup:
        kind, rd = lookup[key][0]
        return True, "exact", 100, kind, rd
    # fuzzy
    keys = list(lookup.keys())
    if not keys:
        return False, None, 0, None, None
    match = process.extractOne(key, keys, scorer=fuzz.token_sort_ratio)
    if match:
        matched_key, score, _ = match
        if score >= FUZZY_THRESHOLD:
            kind, rd = lookup[matched_key][0]
            return True, "fuzzy", int(score), kind, rd
    return False, None, 0, None, None

results = {}
rows = []

for g in tqdm(genes, desc="Annotating genes (Panglao + CellMarker)"):
    # try Panglao first
    ok, mtype, score, kind, rd = match_in_lookup(g, panglao_lookup)
    if ok:
        desc = extract_short_desc_from_row(rd, P_COL_DESC, P_COL_CELL)
        celltype = rd.get(P_COL_CELL, "")
        results[g] = {"source":"PanglaoDB","match_type":mtype,"score":score,"matched_col":kind,"cell_type":celltype,"desc":desc}
        rows.append({"gene":g,"source":"PanglaoDB","match_type":mtype,"score":score,"matched_col":kind,"cell_type":celltype,"description":desc})
        continue
    # fallback to CellMarker
    ok2, mtype2, score2, kind2, rd2 = match_in_lookup(g, cellmarker_lookup)
    if ok2:
        desc = extract_short_desc_from_row(rd2, C_COL_DESC, C_COL_CELL)
        celltype = rd2.get(C_COL_CELL, "")
        results[g] = {"source":"CellMarker","match_type":mtype2,"score":score2,"matched_col":kind2,"cell_type":celltype,"desc":desc}
        rows.append({"gene":g,"source":"CellMarker","match_type":mtype2,"score":score2,"matched_col":kind2,"cell_type":celltype,"description":desc})
        continue
    # not found
    results[g] = {"source":None,"match_type":"none","score":0,"matched_col":None,"cell_type":"","desc":""}
    rows.append({"gene":g,"source":None,"match_type":"none","score":0,"matched_col":None,"cell_type":"","description":""})

# save outputs
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

pd.DataFrame(rows).to_csv(OUT_REVIEW, index=False)
print("Wrote:", OUT_JSON, OUT_REVIEW)


Annotating genes (Panglao + CellMarker): 100%|██████████| 480/480 [00:00<00:00, 1378.42it/s]

Wrote: Data_processing_output/RAG_output/gene_knowledge.json Data_processing_output/RAG_output/gene_knowledge.csv


In [ ]:
import pandas as pd

# Load review table
df = pd.DataFrame(rows)   

total = df.shape[0]

panglao_hits = df[df["source"] == "PanglaoDB"].shape[0]
cellmarker_hits = df[df["source"] == "CellMarker"].shape[0]
no_hits = df[df["source"].isna() | (df["source"] == "None")].shape[0]

print("Total genes:", total)
print("Matched in PanglaoDB:", panglao_hits)
print("Matched in CellMarker:", cellmarker_hits)
print("Total Matched:", panglao_hits + cellmarker_hits)
print("Unmatched:", no_hits)


Total genes: 480
Matched in PanglaoDB: 370
Matched in CellMarker: 85
Total Matched: 455
Unmatched: 25


In [11]:
import pandas as pd

df = pd.DataFrame(rows)

unmatched_df = df[(df["source"].isna()) | (df["source"]=="None")].copy()

print("Unmatched gene count:", unmatched_df.shape[0])
unmatched_df.head()
unmatched_genes = unmatched_df["gene"].tolist()
unmatched_genes


Unmatched gene count: 25


['ADH1A',
 'AFM',
 'AKR1C4',
 'ANXA9',
 'APOF',
 'ASPHD2',
 'C4BPA',
 'CDC25A',
 'CFHR5',
 'CYP8B1',
 'EPHB6',
 'F13B',
 'FZD5',
 'GCNT4',
 'HAO1',
 'HAO2',
 'HKDC1',
 'ITIH1',
 'MAP3K5',
 'NUTM2D',
 'OSGIN2',
 'STEAP2',
 'WNK2',
 'ZNF618',
 'ZXDA']

In [12]:
from pathlib import Path

OUT_DIR = Path("Data_processing_output/RAG_output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

UNMATCHED_CSV = OUT_DIR / "unmatched_genes.csv"

unmatched_df.to_csv(UNMATCHED_CSV, index=False)
print("Saved unmatched genes to:", UNMATCHED_CSV)


Saved unmatched genes to: Data_processing_output/RAG_output/unmatched_genes.csv


In [13]:
import pandas as pd
from pathlib import Path
import json

# Load the review CSV you already created
REVIEW_PATH = "Data_processing_output/RAG_output/gene_knowledge.csv"
df = pd.read_csv(REVIEW_PATH)

print("Original shape:", df.shape)
df.head()


Original shape: (480, 7)


,gene,source,match_type,score,matched_col,cell_type,description
0,ABCC4,PanglaoDB,exact,100,symbol,Platelets,ATP binding cassette subfamily C member 4
1,ACE2,PanglaoDB,fuzzy,88,symbol,Basal cells,beta-secretase 2
2,ACSL5,PanglaoDB,exact,100,symbol,Enterocytes,acyl-CoA synthetase long chain family member 5
3,ADAM12,PanglaoDB,exact,100,symbol,Adipocytes,ADAM metallopeptidase domain 12
4,ADH1A,NaN,none,0,NaN,NaN,NaN


Remove Unmatched Genes (25 genes)

In [14]:
# Keep only matched genes
df_matched = df[df["match_type"] != "none"].copy()

print("After removing unmatched genes:")
print("Remaining genes:", df_matched.shape[0])


After removing unmatched genes:
Remaining genes: 455


In [16]:
# Basic text cleaning
df_matched["cell_type"] = df_matched["cell_type"].str.strip()
df_matched["description"] = df_matched["description"].fillna("").str.strip()

# Optional: normalize gene symbols
df_matched["gene"] = df_matched["gene"].str.upper()


In [17]:
OUT_DIR = Path("Data_processing_output/RAG_output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

CLEAN_CSV = OUT_DIR / "gene_knowledge_clean_phase1.csv"
df_matched.to_csv(CLEAN_CSV, index=False)

print("Saved clean CSV:", CLEAN_CSV)


Saved clean CSV: Data_processing_output/RAG_output/gene_knowledge_clean_phase1.csv


In [18]:
gene_knowledge = {}

for _, row in df_matched.iterrows():
    gene_knowledge[row["gene"]] = {
        "source": row["source"],
        "cell_type": row["cell_type"],
        "description": row["description"],
        "match_type": row["match_type"],
        "confidence_score": int(row["score"])
    }

FINAL_JSON = OUT_DIR / "gene_knowledge_clean_phase1.json"
with open(FINAL_JSON, "w", encoding="utf-8") as f:
    json.dump(gene_knowledge, f, indent=2, ensure_ascii=False)

print("Saved final RAG JSON:", FINAL_JSON)


Saved final RAG JSON: Data_processing_output/RAG_output/gene_knowledge_clean_phase1.json
